In [3]:
import os

In [4]:
import polars as pl
import altair as alt

In [5]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [6]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).drop_nulls(
    subset=['datum']
)

In [7]:
df = df.filter(~pl.col("komora_komplet").str.contains('[Ss]enát'))

In [8]:
sorted(df.select("komora_komplet").unique().to_series().to_list())

['FS ČSSR 1969-1971',
 'FS ČSSR 1971-1976',
 'FS ČSSR 1976-1980',
 'Federální shromáždění ČSFR, 1990 – 1992 Sněmovna lidu',
 'Federální shromáždění ČSFR, 1990 – 1992 Sněmovna národů',
 'Federální shromáždění ČSFR, 1990 – 1992 Společné schůze SL a SN',
 'Federální shromáždění ČSFR, 1992 – 1992 Sněmovna lidu',
 'Federální shromáždění ČSFR, 1992 – 1992 Společné schůze SL a SN',
 'Federální shromáždění ČSSR, 1981 – 1986 Sněmovna lidu',
 'Federální shromáždění ČSSR, 1981 – 1986 Sněmovna národů',
 'Federální shromáždění ČSSR, 1981 – 1986 Společné schůze SL a SN',
 'Federální shromáždění ČSSR, 1986 – 1990 Sněmovna lidu',
 'Federální shromáždění ČSSR, 1986 – 1990 Sněmovna národů',
 'Federální shromáždění ČSSR, 1986 – 1990 Společné schůze SL a SN',
 'NS RČS 1920-1925',
 'NS RČS 1925-1929',
 'NS RČS 1929-1935',
 'NS RČS 1935-1938',
 'NS RČS 1948-1954',
 'NS RČS 1954-1960',
 'NS RČS 1960-1964',
 'NS RČS 1964-1968',
 'Národní shromáždění československé / Poslanecká sněmovna1920 – 1925',
 'Národní 

In [9]:
def drahota(retezec):
    drahoty = None 
    try:
        retezec = ''.join([x for x in retezec if x.isalnum() or x == ' '])
    except Exception as e:
        print(e)
        return None
    if "drahota" in retezec.lower():
        drahoty = [x.split(" ")[-1].strip() for x in retezec.split(" drahota") if len(x) > 2]
    return drahoty

In [10]:
test = 'Nejdřív byla Babišova drahota, teď je zas Fialova drahota.'
drahota(test)

['Babišova', 'Fialova']

In [11]:
df_drahota = df.with_columns(
    pl.col('text').map_elements(drahota).alias('drahota')
)

C:\Users\micha\AppData\Local\Temp\ipykernel_37424\4217153621.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_drahota = df.with_columns(


## Grafy + data

In [13]:
def hledej(retezec, doslova=False, frejm=df):
    return frejm.sort(by="datum").filter(
    pl.col('text').str.contains(retezec, literal=doslova)
).group_by_dynamic(
    index_column='datum',every='1y').agg(pl.col('text').unique().len()
).join(
    frejm.sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()), 
    on='datum', how='left'
).with_columns(
    (pl.col('text') / pl.col('text_right')).alias('podil')
).with_columns(
        pl.lit(retezec).alias('co')
)

In [14]:
try:
    os.makedirs("data/drahota")
except:
    pass

In [15]:
alt.Chart(
    hledej(" [Dd]rahot[auoě]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [16]:
alt.Chart(
    hledej(" [Dd]rahot[auoě]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
).save("data/drahota/drahota.svg")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [17]:
hledej(" [Dd]rahot[auoě]").write_csv("data/drahota/drahota.csv")

In [18]:
alt.Chart(
    hledej(" [Ii]nfla[cč]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [55]:
alt.Chart(
    hledej("[Ii]nfla[cč]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [53]:
alt.Chart(
    hledej(" [Ii]nfla[cč]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
).save("data/drahota/inflace.svg")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [20]:
hledej(" [Ii]nfla[cč]").write_csv("data/drahota/inflace.csv")

In [21]:
alt.Chart(
    hledej(r"Fialo\w{1,5} drahot[auoě]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [22]:
alt.Chart(
    hledej(r"Fialo\w{1,5} drahot[auoě]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
).save('data/drahota/fialova_drahota.svg')

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [23]:
hledej(r"Fialo\w{1,5} drahot[auoě]").write_csv("data/drahota/fialova_drahota.csv")

In [24]:
alt.Chart(
    hledej(r"Babiš\w{1,5} drahot[auoě]").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
).save('data/drahota/babisova_drahota.svg')

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [25]:
hledej(r"Babiš\w{1,5} drahot[auoě]").write_csv("data/drahota/babisova_drahota.csv")

## Nejčastější slova předcházející ve sněmovnách slovu drahota (od r. 1918)

In [27]:
df_drahota.explode(
    'drahota'
).group_by(
    'drahota'
).len(
).sort(
    by='len',
    descending=True
).head(50)

drahota,len
str,u32
null,762757
"""Potlesk""",180
"""že""",86
"""poslanců""",73
"""a""",54
"""Babišova""",46
"""Fialova""",40
"""levici""",40
"""je""",32
